## Data Preprocess 1
1. 메인 데이터(재비산먼지 농도) 수집 및 기본 처리
    * API 인증키 에러가 지속되어 csv 파일로 하나씩 다운로드 받음(67개의 파일) -> 구글 드라이브에 원본 파일 업로드함
    * 디렉토리 내의 파일들 모두 불러와 같은 형식으로 하나의 데이터프레임으로 합침
        * `date`에 연도가 붙지 않고 들어오는 데이터의 경우 임의로 붙여서 같은 형태 유지하도록 함
    * '서울시'의 데이터만 활용 -> 메인 데이터프레임 별도 파일로 저장

In [1]:
# Load libraries
import os
import re
import numpy as np
import pandas as pd
import requests
import pprint
import json
import urllib.request 
import bs4
from lxml import html
from pandas.io.json import json_normalize

### 1. 메인 데이터(재비산먼지 농도) 수집 및 기본 처리

In [23]:
dirname = 'dust_data/'

In [24]:
filenames = os.listdir(dirname)
full_df = []
# 컬럼의 순서는 같지만, 이름이 달라 모두 같은 형식으로 통일
columns = ['date', 'time', 'city', 'gu', 'road', 'start', 'end', 'temperature', 'humidity', 'avg_re_dust', 'dust_ratio']
for k, filename in enumerate(filenames):
    full_filename = os.path.join(dirname, filename)
    df = pd.read_csv(full_filename, encoding='cp949')
    df.columns = columns
    # `date` 형식이 다른 파일이 있어, 해당 파일은 전처리 후 합침
    if len(df['date'][0]) != 10:
        year = re.search(r"\d{4}", filename).group()
        df['date'] = df['date'].apply(lambda x: year+'-'+x)
    if k == 0:
        # 첫 파일의 경우 합칠 데이터가 없어 그대로 데이터프레임 반환
        full_df = df
    else:
        full_df = pd.concat([full_df, df], axis=0)

In [25]:
# 서울시 데이터만 활용할 예정
seoul_df = full_df[full_df['city']=='서울']

In [26]:
# 별도의 파일로 저장
seoul_df.to_csv('../data/seoul_re_dust.csv')

In [27]:
seoul_df.head()

,date,time,city,gu,road,start,end,temperature,humidity,avg_re_dust,dust_ratio
0,2018-05-03,09:50:00,서울,강서구,개화동로,서울특별시 강서구 개화동 659천,서울특별시 강서구 방화동 886-15 도,15.0,46.0,5.0,매우좋음
1,2018-05-03,10:13:00,서울,강서구,공항대로,서울특별시 강서구 방화동 621-4도,서울특별시 양천구 목동 560도,15.0,45.0,20.0,매우좋음
2,2018-05-03,10:41:00,서울,강서구,양천로,서울특별시 강서구 염창동 259-8제,서울특별시 강서구 방화동 515-4도,15.0,47.0,39.0,매우좋음
3,2018-05-03,10:46:00,서울,강서구,초원로,서울특별시 강서구 방화동 651-1도,서울특별시 강서구 방화동 217-156 도,15.0,48.0,13.0,매우좋음
4,2018-05-03,10:50:00,서울,강서구,마곡중앙5로,서울특별시 강서구 방화동 217-156 도,서울특별시 강서구 마곡동 278-6답,15.0,48.0,25.0,매우좋음


### 2. 서울시 생활인구 데이터 수집 및 처리

In [2]:
import dotenv

dotenv.load_dotenv()

True

In [3]:
# seoul_df = pd.read_csv('../data/seoul_re_dust.csv', index_col=0)

In [4]:
# seoul_df['date_cd'] = seoul_df['date'].apply(lambda x: "".join(x.split('-')))

In [ ]:
# from tqdm import tqdm

# living_df = []
# for idx, date_cd in tqdm(enumerate(seoul_df['date_cd'].unique())):
#     living_api = f'http://openapi.seoul.go.kr:8088/{os.environ["living_api"]}/xml/SPOP_LOCAL_RESD_JACHI/1/600/{date_cd}'
#     response = requests.get(living_api)
#     contents = response.text

#     xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
#     rows = xml_obj.findAll('row')
#     # 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
#     row_list = [] # 행값
#     name_list = [] # 열이름값
#     value_list = [] #데이터값

#     # xml 안의 데이터 수집
#     for i in range(0, len(rows)):
#         columns = rows[i].find_all()
#         #첫째 행 데이터 수집
#         for j in range(0,len(columns)):
#             if i ==0:
#                 # 컬럼 이름 값 저장
#                 name_list.append(columns[j].name)
#             # 컬럼의 각 데이터 값 저장
#             value_list.append(columns[j].text)
#         # 각 행의 value값 전체 저장
#         row_list.append(value_list)
#         # 데이터 리스트 값 초기화
#         value_list=[]

#     if idx == 0:
#         living_df = pd.DataFrame(row_list, columns=name_list)
#     else:
#         living_df = pd.concat([living_df, pd.DataFrame(row_list, columns=name_list)], axis=0)

# living_df.to_csv('living_origin.csv')

In [38]:
# living_df = pd.read_csv('living_origin.csv')
# living_df['TOT_LVPOP_CO'] = living_df['TOT_LVPOP_CO'].astype(float)
# living_df_summ = living_df.groupby(['STDR_DE_ID', 'ADSTRD_CODE_SE'])['TOT_LVPOP_CO'].mean()
# living_df_new = living_df_summ.reset_index()
# living_df_new.to_csv('../data/living_df.csv')

### 3. 미세먼지

In [7]:
# from tqdm import tqdm

# fine_dust_df = []
# for idx, date_cd in tqdm(enumerate(seoul_df['date_cd'].unique())):
#     fine_dust_api = f'http://openapi.seoul.go.kr:8088/{os.environ["fine_dust_api"]}/xml/DailyAverageCityAir/1/24/{date_cd}'
#     response = requests.get(fine_dust_api)
#     contents = response.text

#     xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
#     rows = xml_obj.findAll('row')
#     # 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
#     row_list = [] # 행값
#     name_list = [] # 열이름값
#     value_list = [] #데이터값

#     # xml 안의 데이터 수집
#     for i in range(0, len(rows)):
#         columns = rows[i].find_all()
#         #첫째 행 데이터 수집
#         for j in range(0,len(columns)):
#             if i ==0:
#                 # 컬럼 이름 값 저장
#                 name_list.append(columns[j].name)
#             # 컬럼의 각 데이터 값 저장
#             value_list.append(columns[j].text)
#         # 각 행의 value값 전체 저장
#         row_list.append(value_list)
#         # 데이터 리스트 값 초기화
#         value_list=[]

#     if idx == 0:
#         fine_dust_df = pd.DataFrame(row_list, columns=name_list)
#     else:
#         fine_dust_df = pd.concat([fine_dust_df, pd.DataFrame(row_list, columns=name_list)], axis=0)

# fine_dust_df.to_csv('fine_dust_origin.csv')

694it [00:59, 11.66it/s]


In [9]:
# fine_dust_df.to_csv('../data/fine_dust_df.csv')